In [1]:
import os, sys
sys.path.append(os.path.abspath('../src/bin/lib/')) # add path to lib

In [2]:
import yaml
import json
from yaml import CLoader as Loader, CDumper as Dumper
import data_operations as dop
from dotted_dict import DottedDict
from collections import namedtuple
import data_operations as dop
from pandasql import sqldf
import pandas as pds
def pysqldf(q):
    return sqldf(q, globals())

In [3]:
spec_file = "../src/bin/lib/nmdc_data_source.yaml"
with open(spec_file, 'r') as input_file:
    spec = DottedDict(yaml.load(input_file, Loader=Loader))

In [4]:
## put all data in one dataframe; for testing grap 20 rows
# mdf = dop.make_dataframe_from_spec_file (spec_file, nrows=20)
mdf = dop.make_dataframe_from_spec_file (spec_file)
# mdf.head()

In [5]:
## list data sources
mdf.nmdc_data_source.unique()

array(['study_table', 'contact_table', 'proposals_table', 'project_table',
       'project_biosample_table', 'biosample_table', 'ficus_faa_table',
       'ficus_fna_table', 'ficus_fasq_table', 'ficus_jgi_emsl',
       'ficus_emsl'], dtype=object)

In [6]:
## get tables from merged dataframe
study_table = dop.extract_table(mdf, 'study_table')
contact_table = dop.extract_table(mdf, 'contact_table')
proposals_table = dop.extract_table(mdf, 'proposals_table')
project_table = dop.extract_table(mdf, 'project_table')
project_biosample_table = dop.extract_table(mdf, 'project_biosample_table')
biosample_table = dop.extract_table(mdf, 'biosample_table')
faa_table = dop.extract_table(mdf, 'ficus_faa_table')
fna_table = dop.extract_table(mdf, 'ficus_fna_table')
fasq_table = dop.extract_table(mdf, 'ficus_fasq_table')

In [7]:
# study_table.head()
# study_table.gold_study_name
# proposals_table.gold_study

In [10]:
study = dop.make_study_dataframe(study_table, contact_table, proposals_table)
# study.columns
# study.gold_study
# study[['active']] # the [[]] returns a dataframe as opposed to []
# study.gold_id

In [12]:
project = dop.make_project_dataframe(project_table, study_table, contact_table)
# project.columns
# project.head()
# project_table.master_study_id.isin(study.study_id)
# project_table.master_study_id

In [13]:
project_biosample_table = dop.extract_table(mdf, 'project_biosample_table')
biosample_table = dop.extract_table(mdf, 'biosample_table')

In [19]:
biosample = dop.make_biosample_dataframe(biosample_table, project_biosample_table, project_table)

In [21]:
# biosample.head()
len(biosample)
# biosample.columns

1078

In [22]:
faa_table = dop.extract_table(mdf, 'ficus_faa_table')
fna_table = dop.extract_table(mdf, 'ficus_fna_table')
fasq_table = dop.extract_table(mdf, 'ficus_fasq_table')
data_objects_table = pds.concat([faa_table, fna_table, fasq_table], axis=0)
# data_objects.head()

In [26]:
data_objects = dop.make_data_objects_datafame(data_objects_table, project) # NB: using project not project_table
# data_objects.head()